#### Let's take a simple example from Kaggle: https://www.kaggle.com/competitions/titanic/data

# Step 1: Load libraries and dataset

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier

train = pd.read_csv('../titanic/train.csv')
test = pd.read_csv('../titanic/test.csv')
train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


# Step 2: preprocess & feature

In [3]:
# fill missing, extract features
for df in (train, test):
    df['Age'] = df['Age'].fillna(df['Age'].median())
    df['Embarked'] = df['Embarked'].fillna('S')
    df['Fare'] = df['Fare'].fillna(df['Fare'].median())
    df['Title'] = df['Name'].str.extract(r' ([A-Za-z]+)\.')
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1


# one‐hot encode
cols = ['Pclass','Sex','Embarked','Title','FamilySize']
all_df = pd.get_dummies(pd.concat([train[cols], test[cols]]), drop_first=True)
X = all_df.iloc[:len(train)]
X_test = all_df.iloc[len(train):]
y = train['Survived']

In [4]:
all_df

,Pclass,FamilySize,Sex_male,Embarked_Q,Embarked_S,Title_Col,Title_Countess,Title_Don,Title_Dona,Title_Dr,...,Title_Major,Title_Master,Title_Miss,Title_Mlle,Title_Mme,Title_Mr,Title_Mrs,Title_Ms,Title_Rev,Title_Sir
0,3,2,True,False,True,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1,1,2,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
2,3,1,False,False,True,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
3,1,2,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
4,3,1,True,False,True,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,3,1,True,False,True,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
414,1,1,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
415,3,1,True,False,True,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
416,3,1,True,False,True,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False


# Step 3: split, train & evaluate

In [7]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
model = XGBClassifier(eval_metric='logloss')
model.fit(X_train, y_train)
pred = model.predict(X_val)
print('Acc:', accuracy_score(y_val, pred), 'AUC:', roc_auc_score(y_val, model.predict_proba(X_val)[:,1]))

Acc: 0.8100558659217877 AUC: 0.8884813384813385


# Step 4: hyperparameter tuning

In [13]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth':    [3, 5, 7],
    'learning_rate':[0.01, 0.1, 0.2],
    'subsample':    [0.6, 0.8, 1.0]
}
rs = RandomizedSearchCV(
    XGBClassifier(eval_metric='logloss'),
    param_grid, n_iter=10, cv=3,
    scoring='roc_auc', random_state=42, n_jobs=-1
)
rs.fit(X, y)
print(f'Best parameters:\n {rs.best_params_}, \n=============== \nBest Score: \n {rs.best_score_}')

Best parameters:
 {'subsample': 0.8, 'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.1}, 
Best Score: 
 0.869283865401208


# Step 5: train final model & create submission

In [15]:
best = rs.best_estimator_
best.fit(X, y)
submission = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived':    best.predict(X_test)
})
submission.to_csv('../submission.csv', index=False)
submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0
